# imports

In [1]:
import numpy as np
import pandas as pd

# Generate new fitting script from template

In [2]:
#There are three burst where not all detectors are on
#modify the fitting script by hand, since the data grouping will be different

def fit_script(HEDfname=None,KW1fname=None,KW2fname=None,GBMn1fname=None,GBMn2fname=None,GBMb1fname=None,outfname=None):
    
    fnames = [HEDfname,KW1fname,KW2fname,GBMn1fname,GBMn2fname,GBMb1fname,outfname]
    if all(isinstance(fname, str) for fname in fnames) == False:
        return(print("input must be string"))
    
    with open('fitting.xcm',"r") as f:
        lines = f.readlines() #198 lines of code
        for i in np.arange(0,len(lines)):
            lines[i] = lines[i].replace("HED_PHA_1.fits",HEDfname)
            lines[i] = lines[i].replace("KW201227_1_gr20.pha",KW1fname)
            lines[i] = lines[i].replace("KW201227_2_gr20.pha",KW2fname)
            lines[i] = lines[i].replace("GRB201227A_n4_gr20.pha",GBMn1fname)
            lines[i] = lines[i].replace("GRB201227A_n8_gr20.pha",GBMn2fname)
            lines[i] = lines[i].replace("GRB201227A_b0_gr20.pha",GBMb1fname)
            
    with open(outfname+".xcm","w") as out:
        for line in lines:
            out.write(line)

In [3]:
fit_script(HEDfname="bla",KW1fname="blaa",KW2fname="blaaa",GBMn1fname="la",GBMn2fname="laa",GBMb1fname="laaa",outfname="fooo")

In [4]:
#GRB 190206A don't have GBM data, so i can't use the fit_script above
#I can go in an replace HED and KW files, but then i need to remove the GBM grouping and modeling that includes GBM

# Get spectral parameters from log file
    -'Current model list' starts from index 144
    -'\n' contained in every line

In [508]:
with open('template.log','r') as f:
    logLines = f.readlines()
    #looking for this line: #Model constant<1>*cflux<2>*ngrbep<3> Source No.: 1   Active/On
    for l in logLines:
        if l == '#Model constant<1>*cflux<2>*ngrbep<3> Source No.: 1   Active/On':
            print(l)

In [509]:
for i,l in enumerate(logLines):
    if l == '#Current model list:\n':
        print('model list start at index: ', i)
        modList = logLines[i:]
        break

model list start at index:  144


In [7]:
#TODO: load the spec parameters for all the datagroups
#remove '\n' from line

In [8]:
clean_modList = []
for l in modList:
    l = l.replace('\n','')
    l = l.replace('#','')
    clean_modList.append(l)

In [9]:
keys = {}
for i,l in enumerate(clean_modList):
    if 'Data group:' in l:
        key = l.strip()
        keys[key] = i
startingIndecies = np.fromiter(keys.values(), dtype=int)
n_dataGroups = len(startingIndecies)

if np.all(np.diff(startingIndecies)) == True:
    size_dataGroup = np.diff(np.fromiter(keys.values(), dtype=int))[0]
else:
    print('varying length of datagroups')

In [10]:
startingIndecies,n_dataGroups

(array([ 6, 16, 26, 36]), 4)

In [42]:
spec_data = {}
for key,value in keys.items():
    newTargetModel = []
    targetModel = clean_modList[value+1:value+size_dataGroup]
    for par in targetModel:
        newpar = par.strip()
        newpar = newpar.split()
        del newpar[1]
        newTargetModel.append(newpar)
    spec_data[key] = newTargetModel

In [18]:
for l in spec_data.values():
    del l[0][0]
    print(l[0][0])

constant
constant
constant
constant


### Spec values from dict

In [43]:
list(spec_data.values())

[[['1', 'constant', 'factor', '1.27737', '+/-', '0.127300'],
  ['2', 'cflux', 'Emin', 'keV', '10.0000', 'frozen'],
  ['3', 'cflux', 'Emax', 'keV', '1.00000E+04', 'frozen'],
  ['4', 'cflux', 'lg10Flux', 'cgs', '-4.18518', '+/-', '3.00161E-02'],
  ['5', 'ngrbep', 'alpha', '-0.185890', '+/-', '7.28931E-02'],
  ['6', 'ngrbep', 'beta', '-3.44617', '+/-', '0.512888'],
  ['7', 'ngrbep', 'epeak', 'keV', '922.399', '+/-', '58.2440'],
  ['8', 'ngrbep', 'enorm', 'keV', '100.000', 'frozen'],
  ['9', 'ngrbep', 'norm', '1.00000', 'frozen']],
 [['10', 'constant', 'factor', '1.00000', 'frozen'],
  ['11', 'cflux', 'Emin', 'keV', '10.0000', '=', 'p2'],
  ['12', 'cflux', 'Emax', 'keV', '1.00000E+04', '=', 'p3'],
  ['13', 'cflux', 'lg10Flux', 'cgs', '-4.18518', '=', 'p4'],
  ['14', 'ngrbep', 'alpha', '-0.185890', '=', 'p5'],
  ['15', 'ngrbep', 'beta', '-3.44617', '=', 'p6'],
  ['16', 'ngrbep', 'epeak', 'keV', '922.399', '=', 'p7'],
  ['17', 'ngrbep', 'enorm', 'keV', '100.000', '=', 'p8'],
  ['18', 'ngrbep

In [44]:
v = list(spec_data.values())
for i in range(0,4):
    for j in range(0,9):
        del v[i][j][0]
v

[[['constant', 'factor', '1.27737', '+/-', '0.127300'],
  ['cflux', 'Emin', 'keV', '10.0000', 'frozen'],
  ['cflux', 'Emax', 'keV', '1.00000E+04', 'frozen'],
  ['cflux', 'lg10Flux', 'cgs', '-4.18518', '+/-', '3.00161E-02'],
  ['ngrbep', 'alpha', '-0.185890', '+/-', '7.28931E-02'],
  ['ngrbep', 'beta', '-3.44617', '+/-', '0.512888'],
  ['ngrbep', 'epeak', 'keV', '922.399', '+/-', '58.2440'],
  ['ngrbep', 'enorm', 'keV', '100.000', 'frozen'],
  ['ngrbep', 'norm', '1.00000', 'frozen']],
 [['constant', 'factor', '1.00000', 'frozen'],
  ['cflux', 'Emin', 'keV', '10.0000', '=', 'p2'],
  ['cflux', 'Emax', 'keV', '1.00000E+04', '=', 'p3'],
  ['cflux', 'lg10Flux', 'cgs', '-4.18518', '=', 'p4'],
  ['ngrbep', 'alpha', '-0.185890', '=', 'p5'],
  ['ngrbep', 'beta', '-3.44617', '=', 'p6'],
  ['ngrbep', 'epeak', 'keV', '922.399', '=', 'p7'],
  ['ngrbep', 'enorm', 'keV', '100.000', '=', 'p8'],
  ['ngrbep', 'norm', '1.00000', '=', 'p9']],
 [['constant', 'factor', '1.31508', '+/-', '8.29398E-02'],
  ['c

### clean_dict

In [52]:
for i in range(0,4):
    key = list(spec_data.keys())[i]
    clean_dict[key] = v[i]

In [80]:
#Need to get the parameter names and put them where the numbers are in the df
headerName_list = []
for j in range(0,9):
    par = list(clean_dict.values())[0][j][0]
    typ = list(clean_dict.values())[0][j][1]
    header_name = par + ' ' + '[' + typ + ']'
    headerName_list.append(header_name)

In [98]:
#problem that the lists are different lengths

In [ ]:
list(clean_dict.values())[0]

[['constant', 'factor', '1.27737', '+/-', '0.127300'],
 ['cflux', 'Emin', 'keV', '10.0000', 'frozen'],
 ['cflux', 'Emax', 'keV', '1.00000E+04', 'frozen'],
 ['cflux', 'lg10Flux', 'cgs', '-4.18518', '+/-', '3.00161E-02'],
 ['ngrbep', 'alpha', '-0.185890', '+/-', '7.28931E-02'],
 ['ngrbep', 'beta', '-3.44617', '+/-', '0.512888'],
 ['ngrbep', 'epeak', 'keV', '922.399', '+/-', '58.2440'],
 ['ngrbep', 'enorm', 'keV', '100.000', 'frozen'],
 ['ngrbep', 'norm', '1.00000', 'frozen']]

In [290]:
list(clean_dict.values())[0][0]

['constant', 'factor', '1.27737', '+/-', '0.127300']

In [291]:
for i in range(0,9): print(get_cellInput(list(clean_dict.values())[0][i]))

1.277 +/- 0.127
10.0
10000.0
-4.185 +/- 0.03
-0.186 +/- 0.073
-3.446 +/- 0.513
922.399 +/- 58.244
100.0
1.0


In [297]:
df.values[0][0] = '1.277 +/- 0.127'

In [298]:
df.values[0][0]

'1.277 +/- 0.127'

In [285]:
def get_cellInput(l):
    cell_input = ''
    for x in l:
        if x == '+/-':
            cell_input += ' ' + x + ' '
        else:
            try:
                float(x)
                cell_input += str(np.round(float(x),3))
            except ValueError:
                continue
    return(cell_input)

In [300]:
for i in range(0,4):
    for j in range(0,9):
        #print(list(clean_dict.values())[i][j])
        df.values[i][j] = (get_cellInput(list(clean_dict.values())[0][i]))

[['1.395 +/- 0.093', '10.0', '10000.0', '-4.185', '-0.186', '-3.446', '922.399', '100.0', '1.0'], ['1.395 +/- 0.093', '10.0', '10000.0', '-4.185', '-0.186', '-3.446', '922.399', '100.0', '1.0'], ['1.395 +/- 0.093', '10.0', '10000.0', '-4.185', '-0.186', '-3.446', '922.399', '100.0', '1.0'], ['1.395 +/- 0.093', '10.0', '10000.0', '-4.185', '-0.186', '-3.446', '922.399', '100.0', '1.0']]
[['1.395 +/- 0.093', '10.0', '10000.0', '-4.185', '-0.186', '-3.446', '922.399', '100.0', '1.0'], ['1.395 +/- 0.093', '10.0', '10000.0', '-4.185', '-0.186', '-3.446', '922.399', '100.0', '1.0'], ['1.395 +/- 0.093', '10.0', '10000.0', '-4.185', '-0.186', '-3.446', '922.399', '100.0', '1.0'], ['1.395 +/- 0.093', '10.0', '10000.0', '-4.185', '-0.186', '-3.446', '922.399', '100.0', '1.0']]
[['1.395 +/- 0.093', '10.0', '10000.0', '-4.185', '-0.186', '-3.446', '922.399', '100.0', '1.0'], ['1.395 +/- 0.093', '10.0', '10000.0', '-4.185', '-0.186', '-3.446', '922.399', '100.0', '1.0'], ['1.395 +/- 0.093', '10.0',

In [301]:
df

,constant [factor],cflux [Emin],cflux [Emax],cflux [lg10Flux],ngrbep [alpha],ngrbep [beta],ngrbep [epeak],ngrbep [enorm],ngrbep [norm]
Data group: 1,1.277 +/- 0.127,1.277 +/- 0.127,1.277 +/- 0.127,1.277 +/- 0.127,1.277 +/- 0.127,1.277 +/- 0.127,1.277 +/- 0.127,1.277 +/- 0.127,1.277 +/- 0.127
Data group: 2,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
Data group: 3,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0
Data group: 4,-4.185 +/- 0.03,-4.185 +/- 0.03,-4.185 +/- 0.03,-4.185 +/- 0.03,-4.185 +/- 0.03,-4.185 +/- 0.03,-4.185 +/- 0.03,-4.185 +/- 0.03,-4.185 +/- 0.03


In [277]:
#Problem assigning cell input into the cell_input list

In [347]:
df = pd.DataFrame(data = clean_dict).T
df

,0,1,2,3,4,5,6,7,8
Data group: 1,"[factor, 1.27737, +/-, 0.127300]","[cflux, Emin, keV, 10.0000, frozen]","[cflux, Emax, keV, 1.00000E+04, frozen]","[cflux, lg10Flux, cgs, -4.18518, +/-, 3.00161E...","[ngrbep, alpha, -0.185890, +/-, 7.28931E-02]","[ngrbep, beta, -3.44617, +/-, 0.512888]","[ngrbep, epeak, keV, 922.399, +/-, 58.2440]","[ngrbep, enorm, keV, 100.000, frozen]","[ngrbep, norm, 1.00000, frozen]"
Data group: 2,"[factor, 1.00000, frozen]","[cflux, Emin, keV, 10.0000, =, p2]","[cflux, Emax, keV, 1.00000E+04, =, p3]","[cflux, lg10Flux, cgs, -4.18518, =, p4]","[ngrbep, alpha, -0.185890, =, p5]","[ngrbep, beta, -3.44617, =, p6]","[ngrbep, epeak, keV, 922.399, =, p7]","[ngrbep, enorm, keV, 100.000, =, p8]","[ngrbep, norm, 1.00000, =, p9]"
Data group: 3,"[factor, 1.31508, +/-, 8.29398E-02]","[cflux, Emin, keV, 10.0000, =, p2]","[cflux, Emax, keV, 1.00000E+04, =, p3]","[cflux, lg10Flux, cgs, -4.18518, =, p4]","[ngrbep, alpha, -0.185890, =, p5]","[ngrbep, beta, -3.44617, =, p6]","[ngrbep, epeak, keV, 922.399, =, p7]","[ngrbep, enorm, keV, 100.000, =, p8]","[ngrbep, norm, 1.00000, =, p9]"
Data group: 4,"[factor, 1.39527, +/-, 9.27995E-02]","[cflux, Emin, keV, 10.0000, =, p2]","[cflux, Emax, keV, 1.00000E+04, =, p3]","[cflux, lg10Flux, cgs, -4.18518, =, p4]","[ngrbep, alpha, -0.185890, =, p5]","[ngrbep, beta, -3.44617, =, p6]","[ngrbep, epeak, keV, 922.399, =, p7]","[ngrbep, enorm, keV, 100.000, =, p8]","[ngrbep, norm, 1.00000, =, p9]"


In [258]:
for i,key in enumerate(list(df.columns)):
    df.rename({key: headerName_list[i]}, axis=1,inplace=True)

In [260]:
list(keys.keys())

['Data group: 1', 'Data group: 2', 'Data group: 3', 'Data group: 4']

In [303]:
print(df.T.style.to_latex(position_float='centering',position='h!',hrules=True,multirow_align='c'))

\begin{table}[h!]
\centering
\begin{tabular}{lllll}
\toprule
 & Data group: 1 & Data group: 2 & Data group: 3 & Data group: 4 \\
\midrule
constant [factor] & 1.277 +/- 0.127 & 10.0 & 10000.0 & -4.185 +/- 0.03 \\
cflux [Emin] & 1.277 +/- 0.127 & 10.0 & 10000.0 & -4.185 +/- 0.03 \\
cflux [Emax] & 1.277 +/- 0.127 & 10.0 & 10000.0 & -4.185 +/- 0.03 \\
cflux [lg10Flux] & 1.277 +/- 0.127 & 10.0 & 10000.0 & -4.185 +/- 0.03 \\
ngrbep [alpha] & 1.277 +/- 0.127 & 10.0 & 10000.0 & -4.185 +/- 0.03 \\
ngrbep [beta] & 1.277 +/- 0.127 & 10.0 & 10000.0 & -4.185 +/- 0.03 \\
ngrbep [epeak] & 1.277 +/- 0.127 & 10.0 & 10000.0 & -4.185 +/- 0.03 \\
ngrbep [enorm] & 1.277 +/- 0.127 & 10.0 & 10000.0 & -4.185 +/- 0.03 \\
ngrbep [norm] & 1.277 +/- 0.127 & 10.0 & 10000.0 & -4.185 +/- 0.03 \\
\bottomrule
\end{tabular}
\end{table}



# final function

In [475]:
def get_datagroups(log_fname=None):
    if type(log_fname) != str:
        return print('log_fname must be str')
    else:
        with open(log_fname,'r') as f:
            
            logLines = f.readlines()
            #looking for this line: #Model constant<1>*cflux<2>*ngrbep<3> Source No.: 1   Active/On
            for l in logLines:
                if l == '#Model constant<1>*cflux<2>*ngrbep<3> Source No.: 1   Active/On':
                    print(l)

            for i,l in enumerate(logLines):
                if l == '#Current model list:\n':
                    print('model list start at index: ', i)
                    modList = logLines[i:]
                    break
                    
    clean_modList = []
    for l in modList:
        l = l.replace('\n','')
        l = l.replace('#','')
        clean_modList.append(l)

    keys = {}
    for i,l in enumerate(clean_modList):
        if 'Data group:' in l:
            key = l.strip()
            keys[key] = i
    startingIndecies = np.fromiter(keys.values(), dtype=int)
    n_dataGroups = len(startingIndecies)

    if np.all(np.diff(startingIndecies)) == True:
        size_dataGroup = np.diff(np.fromiter(keys.values(), dtype=int))[0]
    else:
        print('varying length of datagroups')
        
    print('startingIndecies,n_dataGroups: ',startingIndecies,n_dataGroups)
    spec_data = {}
    for key,value in keys.items():
        newTargetModel = []
        targetModel = clean_modList[value+1:value+size_dataGroup]
        for par in targetModel:
            newpar = par.strip()
            newpar = newpar.split()
            del newpar[1]
            newTargetModel.append(newpar)
        spec_data[key] = newTargetModel
        
    for l in spec_data.values(): #deleting 'constant' keyword
        del l[0][0]
        
    v = list(spec_data.values())
    #TODO: replace  4 with 'n datagroups' & 9 with 
    for i in range(0,4): #replace n-1 -diff(n_datagroups)
        for j in range(0,9):
            del v[i][j][0]

    for i in range(0,4):#replace 4
        key = list(spec_data.keys())[i]
        clean_dict[key] = v[i] 
        
    df = pd.DataFrame(data = clean_dict).T #this works
    
    #TODO: set the column names
    headerName_list = []
    for j in range(0,9):
        par = list(clean_dict.values())[0][j][0]
        typ = list(clean_dict.values())[0][j][1]
        header_name = par + ' ' + '[' + typ + ']'
        headerName_list.append(header_name)
           
    for i,l in enumerate(headerName_list): #could also change Emin to have keV...
        if 'factor' in l.split():
            headerName_list[i] = 'factor [constant]'
     
    for i,key in enumerate(list(df.columns)):
        df.rename({key: headerName_list[i]}, axis=1,inplace=True)
        
    for i in range(0,4):
        for j in range(0,9):
            df.values[i][j] = (get_cellInput(list(clean_dict.values())[i][j]))
    return df


In [452]:
from timeit import default_timer as timer

In [503]:
start = timer()
df = (get_datagroups('template.log'))
end = timer()
print(end - start)

model list start at index:  144
startingIndecies,n_dataGroups:  [ 6 16 26 36] 4
0.009199459000228671


In [477]:
df

,factor [constant],cflux [Emin],cflux [Emax],cflux [lg10Flux],ngrbep [alpha],ngrbep [beta],ngrbep [epeak],ngrbep [enorm],ngrbep [norm]
Data group: 1,1.277 +/- 0.127,10.0,10000.0,-4.185 +/- 0.03,-0.186 +/- 0.073,-3.446 +/- 0.513,922.399 +/- 58.244,100.0,1.0
Data group: 2,1.0,10.0,10000.0,-4.185,-0.186,-3.446,922.399,100.0,1.0
Data group: 3,1.315 +/- 0.083,10.0,10000.0,-4.185,-0.186,-3.446,922.399,100.0,1.0
Data group: 4,1.395 +/- 0.093,10.0,10000.0,-4.185,-0.186,-3.446,922.399,100.0,1.0


In [342]:
def df_toLatex(df=None):
    import pandas as pd
    
    if type(df) != pd.core.frame.DataFrame:
        return print('input df not correct format')
    else:
        print(df.T.style.to_latex(position_float='centering',position='h!',hrules=True,multirow_align='c'))

In [504]:
df.T

,Data group: 1,Data group: 2,Data group: 3,Data group: 4
factor [constant],1.277 +/- 0.127,1.0,1.315 +/- 0.083,1.395 +/- 0.093
cflux [Emin],10.0,10.0,10.0,10.0
cflux [Emax],10000.0,10000.0,10000.0,10000.0
cflux [lg10Flux],-4.185 +/- 0.03,-4.185,-4.185,-4.185
ngrbep [alpha],-0.186 +/- 0.073,-0.186,-0.186,-0.186
ngrbep [beta],-3.446 +/- 0.513,-3.446,-3.446,-3.446
ngrbep [epeak],922.399 +/- 58.244,922.399,922.399,922.399
ngrbep [enorm],100.0,100.0,100.0,100.0
ngrbep [norm],1.0,1.0,1.0,1.0


In [506]:
df_toLatex(df=df) #still the same problem of having the same row for all columns

\begin{table}[h!]
\centering
\begin{tabular}{lllll}
\toprule
 & Data group: 1 & Data group: 2 & Data group: 3 & Data group: 4 \\
\midrule
factor [constant] & 1.277 +/- 0.127 & 1.0 & 1.315 +/- 0.083 & 1.395 +/- 0.093 \\
cflux [Emin] & 10.0 & 10.0 & 10.0 & 10.0 \\
cflux [Emax] & 10000.0 & 10000.0 & 10000.0 & 10000.0 \\
cflux [lg10Flux] & -4.185 +/- 0.03 & -4.185 & -4.185 & -4.185 \\
ngrbep [alpha] & -0.186 +/- 0.073 & -0.186 & -0.186 & -0.186 \\
ngrbep [beta] & -3.446 +/- 0.513 & -3.446 & -3.446 & -3.446 \\
ngrbep [epeak] & 922.399 +/- 58.244 & 922.399 & 922.399 & 922.399 \\
ngrbep [enorm] & 100.0 & 100.0 & 100.0 & 100.0 \\
ngrbep [norm] & 1.0 & 1.0 & 1.0 & 1.0 \\
\bottomrule
\end{tabular}
\end{table}



# Statistics

In [704]:
def get_statistics(log_lines=None):
    #TODO: Read the lines and find where the statistics is given
    #Want Total fit statistics, null hypothesis
    for i,l in enumerate(log_lines):
        line = l.strip()
        if 'Total fit statistic' in line:
            temp_totStatistic = line.replace('#','').split()
            totStatistic = ' '.join(temp_totStatistic)
            
        #TODO: get the errors in the same loop, so i don't have to run it twice    
        elif '!XSPEC12>error 1.' in line:
            err_command = line[17:].split() 
            N_parameters = get_Nerr_parameters(err_command)
            err1_lines = log_lines[i+2:i+2+N_parameters]
            #Now read the following N_parameter lines 
            dict_err1 = {'Parameter':[], 'Confidence Range (1)':[]}

            for l in err1_lines:
                #TODO: check if the line is valid
                l = clean_line(l)
                if '***Warning' in l or len(l)==0:
                    print('Reached end of error 1.')
                    break
                else:
                    dict_err1['Parameter'].append(clean_line(l).split()[0])
                    dict_err1['Confidence Range (1)'].append('    '.join(clean_line(l).split()[1:]))
            df_err1 = pd.DataFrame(dict_err1)
            if len(dict_err1['Parameter']) != N_parameters:
                print('Not all parameters calculated for error 1.')
            else:
                print('All parameters calculated for error 1.')
                
        elif '!XSPEC12>error 2.706' in line:
            err_command = line[21:].split() 
            N_parameters = get_Nerr_parameters(err_command)
            err2_lines = log_lines[i+2:i+2+N_parameters]
            #Now read the following N_parameter lines 
            dict_err2 = {'Parameter':[], 'Confidence Range (2.706)':[]}

            for l in err2_lines:
                #TODO: check if the line is valid
                l = clean_line(l)
                if '***Warning' in l or len(l)==0:
                    print('Reached end of error 2.706')
                    break
                else:
                    dict_err2['Parameter'].append(clean_line(l).split()[0])
                    dict_err2['Confidence Range (2.706)'].append('    '.join(clean_line(l).split()[1:]))
                    
            df_err2 = pd.DataFrame(dict_err2)
            if len(dict_err2['Parameter']) != N_parameters:
                print('Not all parameters calculated for error 2.706')
            else:
                print('All parameters calculated for error 2.706')
    
    return totStatistic,df_err1,df_err2
totStatistic,df_err1,df_err2 = get_statistics(log_lines=logLines)


All parameters calculated for error 1.
Reached end of error 2.706
Not all parameters calculated for error 2.706


In [705]:
totStatistic

'Total fit statistic 88.54 with 87 d.o.f.'

In [706]:
df_err1

,Parameter,Confidence Range (1)
0,1,"1.15486 1.41086 (-0.122507,0.133494)"
1,19,"1.23494 1.40194 (-0.0801467,0.0868618)"
2,28,"1.30668 1.49112 (-0.0885832,0.0958577)"
3,4,"-4.21571 -4.1526 (-0.0305203,0.0325901)"
4,5,"-0.255639 -0.106621 (-0.0697578,0.07926)"
5,6,"-4.22559 -3.03039 (-0.779171,0.416028)"
6,7,"860.555 984.489 (-61.8521,62.0822)"


In [707]:
df_err2

,Parameter,Confidence Range (2.706)
0,1,"1.08093 1.50359 (-0.196433,0.226221)"
1,19,"1.18599 1.46183 (-0.129095,0.146752)"
2,28,"1.25263 1.5575 (-0.142635,0.162233)"
3,4,"-4.23552 -4.1351 (-0.0503282,0.0500879)"
4,5,"-0.296555 -0.058995 (-0.110671,0.126889)"


In [709]:
df.T 

,Data group: 1,Data group: 2,Data group: 3,Data group: 4
factor [constant],1.277 +/- 0.127,1.0,1.315 +/- 0.083,1.395 +/- 0.093
cflux [Emin],10.0,10.0,10.0,10.0
cflux [Emax],10000.0,10000.0,10000.0,10000.0
cflux [lg10Flux],-4.185 +/- 0.03,-4.185,-4.185,-4.185
ngrbep [alpha],-0.186 +/- 0.073,-0.186,-0.186,-0.186
ngrbep [beta],-3.446 +/- 0.513,-3.446,-3.446,-3.446
ngrbep [epeak],922.399 +/- 58.244,922.399,922.399,922.399
ngrbep [enorm],100.0,100.0,100.0,100.0
ngrbep [norm],1.0,1.0,1.0,1.0


In [ ]:
#TODO: Excract the par number and insert new column 

### error 1. example

In [693]:
err1_lines = [ '!XSPEC12>error 2.706 1, 19, 28, 4-7\n',
 '# Parameter   Confidence Range (2.706)\n',
 '#     1      1.08093      1.50359    (-0.196433,0.226221)\n',
 '#    19      1.18599      1.46183    (-0.129095,0.146752)\n',
 '#    28      1.25263       1.5575    (-0.142635,0.162233)\n',
 '#     4     -4.23552      -4.1351    (-0.0503282,0.0500879)\n',
 '#     5    -0.296555    -0.058995    (-0.110671,0.126889)\n',
 '#\n',
 '#***Warning: Number of trials exceeded before bracketing of delta fit-stat.\n']
#Now read the following N_parameter lines 
dict_err1 = {'Parameter':[],
            'Confidence Range (1)':[]}

for l in err1_lines:
    #TODO: check if the line is valid
    l = clean_line(l)
    if '***Warning' in l or len(l)==0:
        print('Reached end of error lines')
        break
    else:
        dict_err1['Parameter'].append(clean_line(l).split()[0])
        dict_err1['Confidence Range (1)'].append('    '.join(clean_line(l).split()[1:]))
df_err1 = pd.DataFrame(dict_err1)
df_err1

Reached end of error lines


,Parameter,Confidence Range (1)
0,!XSPEC12>error,"2.706 1, 19, 28, 4-7"
1,Parameter,Confidence Range (2.706)
2,1,"1.08093 1.50359 (-0.196433,0.226221)"
3,19,"1.18599 1.46183 (-0.129095,0.146752)"
4,28,"1.25263 1.5575 (-0.142635,0.162233)"
5,4,"-4.23552 -4.1351 (-0.0503282,0.0500879)"
6,5,"-0.296555 -0.058995 (-0.110671,0.126889)"


In [680]:
def clean_line(line=None):
    l = line.replace('#','').strip()
    return l.replace('\n','')

In [585]:
def get_Nerr_parameters(err_command):
    '''function for calculating how many parameters the err_command asks to calculate'''
    
    i=0
    for x in err_command:
        if '-' not in x:
            try:
                int(x.replace(',',''))
                i += 1
            except ValueError:
                print('parameter not defined correctly in error command', x)
        else:
            try:
                err_series = x.split('-')
                n_par_series = int(err_series[-1])-int(err_series[0])+1 #+1 because last index is included
                i += n_par_series
            except ValueError:
                print('parameter not defined correctly in error command', x)
    return i

In [550]:
#TOREAD: Find Satoshi paper for how to format table containing parameters

In [302]:
#see this doc for formatting the table further
# https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.format.html#pandas.io.formats.style.Styler.format 
